# 公開データを整理する
公開するデータを整理するタスクです。<br>
公開したいデータをコピーして、整理を行って下さい。


## 公開データを整理する
以下の手順に従って公開対象となるデータの整理を行ってください。

### 1. データをコピーする
公開対象のデータを「公開データ保存フォルダ」にコピーして下さい。コピー方法は以下の二通りがあります。<br>

#### 1-1. 特定のファイルを自動でコピーする
「データをコピーする」セルを実行して表示されるボタンを押下することで、それぞれ「ガバナンスシート」、「メタデータ」、「検証結果」、「実行済みタスクのノートブック」を「公開データ保存フォルダ」にコピーします。<br>
上述のうち最初の３つはGakuNin RDM上に保存されているものをコピーします。また、「ガバナンスシート」は「ガバナンスシートを登録する」タスク、「メタデータ」は「メタデータを登録する」タスク、「検証結果」は「ガバナンス状態を検証する」タスクを実行することで作成されます。<br>
「実行済みタスクのノートブック」はこのサブフローの親となっている全サブフローの実行済みタスクのノートブックを指します。<br>

#### 1-1. 親サブフローのデータを手動でコピーする
「データをコピーする」セルを実行することで親サブフローのフォルダとデータのコピーを保存するためのフォルダを表示することができます。<br>
以下の図の説明に従って公開したいデータのコピーを行ってください。<br>

図１ 親サブフローのフォルダー<br>
![データをダウンロードする](./images/RF007006_download.png)<br>
図２ データのコピーを保存するためのフォルダ<br>
![データをアップロードする](./images/RF007006_upload.png)<br>
親サブフローを変更する場合はメインメニューに戻り親子関係を変更した後、このタスクを実行してください。<br>
リサーチフロー上に保存していないデータの中に公開したいデータがある場合はこのタスクで追加してください。<br>

In [ ]:
# データをコピーする
import json
import os
import traceback
from typing import Optional

from IPython.core.display import Javascript
import panel as pn
from requests.exceptions import RequestException

from library.task_director import TaskDirector
from library.utils.access import open_data_folder
from library.utils.config import path_config, message as msg_config, connect as con_config
from library.utils.error import (UnusableVault, ProjectNotExist, NotFoundSubflowDataError,
                                    UnauthorizedError, RepoPermissionError)
from library.utils.file import JsonFile
from library.utils.input import get_grdm_connection_parameters
from library.main_menu.subflow_controller import utils
from library.utils.setting import get_data_dir, SubflowStatusFile
from library.utils.setting.research_flow_status import get_subflow_type_and_id, ResearchFlowStatusOperater
from library.utils.storage_provider import grdm
from library.utils.widgets import MessageBox, Button


notebook_name = 'organize_public_data.ipynb'

class CreatePublicFolder(TaskDirector):
    """公開用のデータフォルダを作成するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            _msg_output(MessageBox): メッセージ出力用のボックス
            _form_section(pn.WidgetBox): ボタン等の出力を格納するためのボックス
            abs_root(str): 実行Notebookファイルパスから取得した絶対パス
            reserch_flow_status_operater(ResearchFlowStatusOperater): リサーチフロー操作クラスのインスタンス
            grdm_url(str): GakuNin RDMのベースURL
            grdm(Grdm): GakuNin RDMの操作クラスのインスタンス
            experiment_subflow(dict): 実験サブフローの情報を格納する辞書
            writing_subflow(dict): 論文執筆サブフローの情報を格納する辞書
            review_subflow(dict): 査読サブフローの情報を格納する辞書

    """
    def __init__(self, working_path: str) -> None:
        """Displyクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

        self.abs_root = path_config.get_abs_root_form_working_dg_file_path(self.working_path)
        research_flow_status_file_path = path_config.get_research_flow_status_file_path(self.abs_root)
        self.reserch_flow_status_operater = ResearchFlowStatusOperater(research_flow_status_file_path)

        self.grdm_url = con_config.get('GRDM', 'BASE_URL')
        self.grdm = grdm.Grdm()

        # メッセージボックスの設定
        self._msg_output = MessageBox()
        self._msg_output.width = 900
        # サブフローエラー用のメッセージボックスの設定
        self._msg_subflow = MessageBox()
        self._msg_subflow.width = 900
        # 出力用フォームの設定
        self._form_section = pn.WidgetBox()
        # フェーズごとにサブフローを管理する辞書
        self.experiment_subflow = {}
        self.writing_subflow = {}
        self.review_subflow = {}

    def get_grdm_params(self) -> tuple[str, str]:
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。

        Returns:
            str:GRDMのトークンの値を返す。
            str:プロジェクトIDの値を返す。
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters(self.grdm_url)
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RepoPermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("1")
    def generate_buttons(self):
        """データフォルダをの表示とデータのコピーをするためのボタンを生成するメソッドです。"""

        self.doing_task()

        # ガバナンスシート用ボタン
        self.govsheet_button = Button(width=500)
        self.govsheet_button_title = msg_config.get('public_data', 'copy_govsheet')
        self.govsheet_button.set_looks_init(self.govsheet_button_title)
        self.govsheet_button.on_click(self._handle_click_govsheet)

        # メタデータ用ボタン
        self.metadata_button = Button(width=500)
        self.metadata_button_title = msg_config.get('public_data', 'copy_metadata')
        self.metadata_button.set_looks_init(self.metadata_button_title)
        self.metadata_button.on_click(self._handle_click_metadata)
        # 検証結果用
        self.validate_button = Button(width=500)
        self.validate_button_title = msg_config.get('public_data', 'copy_validate')
        self.validate_button.set_looks_init(self.validate_button_title)
        self.validate_button.on_click(self._handle_click_validata)

        # 実行済みノートブック用
        self.notebooks_button = Button(width=500)
        self.notebooks_button_title = msg_config.get('public_data', 'copy_notebooks')
        self.notebooks_button.set_looks_init(self.notebooks_button_title)
        self.notebooks_button.on_click(self.copy_notebooks)

        title = f"## {msg_config.get('public_data', 'copy_auto')}"
        widget_box_auto = pn.WidgetBox(
            title, pn.Spacer(height=20),
            self.govsheet_button, pn.Spacer(height=20),
            self.metadata_button, pn.Spacer(height=20),
            self.validate_button, pn.Spacer(height=20),
            self.notebooks_button, pn.Spacer(height=20)
        )
        self._form_section.append(widget_box_auto)
        widget_box_auto.append(self._msg_output)
        self._form_section.append(pn.Spacer(height=20))

        title = f"## {msg_config.get('public_data', 'copy_manual')}"
        widget_box_manual = pn.WidgetBox(title, pn.Spacer(height=20))

        # 親サブフローフォルダー表示用
        _, self.subflow_id = get_subflow_type_and_id(self.working_path)
        self.get_parent_info(5, self.subflow_id)

        ## 親サブフローが研究準備サブフローしか存在しない場合はメッセージを格納、表示する
        if not self.experiment_subflow and not self.writing_subflow and not self.review_subflow:
            self._msg_subflow.update_warning(msg_config.get("organize_argument_data", "not_found_subflow"))
            widget_box_manual.append(self._msg_subflow)

        else:
            old_part= os.path.join("publication", self.subflow_id)

            ## 取得した実験サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.experiment_subflow.items():
                new_part = os.path.join("experiment", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_experiment')}{parent_name}",
                    width = 550,
                    height = 100
                )
                widget_box_manual.append(card)

            ## 取得した論文執筆サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.writing_subflow.items():
                new_part = os.path.join("writing", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_writing')}{parent_name}",
                    width = 550,
                    height = 100
                )
                widget_box_manual.append(card)

            ## 取得した査読サブフローのフォルダを開くボタンの作成
            for parent_name, parent_id in self.review_subflow.items():
                new_part = os.path.join("review", parent_id)
                new_working_path = self.working_path.replace(old_part, new_part)
                button = open_data_folder(new_working_path)
                card = pn.Card(
                    button,
                    title = f"{msg_config.get('organize_argument_data', 'title_review')}{parent_name}",
                    width = 550,
                    height = 100
                )
                widget_box_manual.append(card)

        ## 公開データを保存するフォルダを開くボタンの作成
        pn.extension(raw_css=[".custom-card { background-color:rgba(255, 255, 0, 0.1);}"])
        button = open_data_folder(self.working_path, 'public_data')
        card = pn.Card(
            button,
            title = f"{msg_config.get('organize_argument_data', 'title_public_data')}",
            width = 550,
            height = 100,
            css_classes = ["custom-card"]
        )
        widget_box_manual.append(card)

        self.done_task()

        pn.extension()
        self._form_section.append(widget_box_manual)
        display(self._form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_parent_info(self, phase_seq_number: int, current_subflow_id: str):
        """親サブフローとなっている実験サブフローを取得するメソッドです。

        Args:
            phase_seq_number(int): 対象のフェーズシーケンス番号
            current_subflow (str): 対象のサブフローのID

        """
        # 全ての親サブフローIDを取得
        parent_ids = self.reserch_flow_status_operater.get_parent_ids(phase_seq_number, current_subflow_id)

        # 全ての親サブフロー名を取得
        phase_seq_number -= 1
        for parent_id in parent_ids:
            # フェーズ１（plan)以外を対象
            while phase_seq_number > 1:
                try:
                    parent_name = self.reserch_flow_status_operater.get_flow_name(phase_seq_number, parent_id)
                    if phase_seq_number == 2:
                        self.experiment_subflow[parent_name] = parent_id
                        break

                    elif phase_seq_number == 3:
                        self.writing_subflow[parent_name] = parent_id

                    elif phase_seq_number == 4:
                        self.review_subflow[parent_name] = parent_id
                    # 全ての親サブフローを取得するように再帰的に呼び出す
                    self.get_parent_info(phase_seq_number, parent_id)
                    break
                # phase_seq_numberが一致しなかった場合は一つ前のフェーズで探索する
                except NotFoundSubflowDataError:
                    phase_seq_number -= 1
        return

    async def _handle_click_govsheet(self, event):
        await self.copy_govsheet(event)

    @TaskDirector.callback_form('ガバナンスシートをコピーする')
    async def copy_govsheet(self, event):
        """ガバナンスシートをコピーするボタンが押下されたときに実行するメソッドです。"""
        try:
            self._msg_output.clear()
            self.govsheet_button.disabled = True
            self.govsheet_button.set_looks_processing(msg_config.get('public_data', 'copy_doing'))
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            #ガバナンスシートのパス
            govsheet_remote_path = con_config.get('DG_WEB', 'GOVSHEET_PATH')
            ## データの取得
            if self._msg_output.has_message():
                self.govsheet_button.set_looks_warning(f"{self.govsheet_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return
            else:
                data = await self.get_data(
                    token=self.token,
                    project_id=self.project_id, remote_path=govsheet_remote_path
                )
            if self._msg_output.has_message():
                self.govsheet_button.set_looks_warning(f"{self.govsheet_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return
            else:
                public_dir_path = os.path.join(get_data_dir(self.working_path), 'public_data')
                file_path = JsonFile(os.path.join(public_dir_path, 'gov-sheet.json'))
                file_path.write(data)
                self.govsheet_button.set_looks_success(f"{self.govsheet_button_title} : {msg_config.get('public_data', 'copy_done')}")
                self.govsheet_button.disabled = False

        except FileNotFoundError:
            message = msg_config.get('dg_web', 'missing_govsheet')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            self.govsheet_button.set_looks_warning(f"{self.govsheet_button_title} : {msg_config.get('public_data', 'copy_failed')}")
            return

    async def _handle_click_metadata(self, event):
        await self.copy_metadata(event)

    @TaskDirector.callback_form('メタデータをコピーする')
    async def copy_metadata(self, event):
        """メタデータをコピーするボタンが押下されたときに実行するメソッドです。"""
        try:
            self._msg_output.clear()
            self.metadata_button.disabled = True
            self.metadata_button.set_looks_processing(msg_config.get('public_data', 'copy_doing'))
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            #メタデータシートのパス
            metadata_remote_path = con_config.get('DG_WEB', 'METADATA_PATH')
            ## データの取得

            if self._msg_output.has_message():
                self.metadata_button.set_looks_warning(f"{self.metadata_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return
            else:
                data = await self.get_data(
                    token=self.token,
                    project_id=self.project_id, remote_path=metadata_remote_path
                )

            if self._msg_output.has_message():
                self.metadata_button.set_looks_warning(f"{self.metadata_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return

            else:
                public_dir_path = os.path.join(get_data_dir(self.working_path), 'public_data')
                file_path = JsonFile(os.path.join(public_dir_path, 'metadata.json'))
                file_path.write(data)
                self.metadata_button.set_looks_success(f"{self.metadata_button_title} : {msg_config.get('public_data', 'copy_done')}")
                self.metadata_button.disabled = False

        except FileNotFoundError:
            message = msg_config.get('dg_web', 'missing_metadata')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            self.metadata_button.set_looks_warning(f"{self.metadata_button_title} : {msg_config.get('public_data', 'copy_failed')}")
            return

    async def _handle_click_validata(self, event):
        await self.copy_validate(event)

    @TaskDirector.callback_form('検証結果をコピーする')
    async def copy_validate(self, event):
        """検証結果をコピーするボタンが押下されたときに実行するメソッドです。"""
        try:
            self._msg_output.clear()
            self.validate_button.disabled = True
            self.validate_button.set_looks_processing(msg_config.get('public_data', 'copy_doing'))
            # パラメータの取得
            self.token, self.project_id = self.get_grdm_params()
            #検証結果のパス
            validate_remote_path = con_config.get('DG_WEB', 'VALIDATE_RESULT')
            ## データの取得
            if self._msg_output.has_message():
                self.validate_button.set_looks_warning(f"{self.validate_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return
            else:
                data = await self.get_data(
                    token=self.token,
                    project_id=self.project_id, remote_path=validate_remote_path
                )

            if self._msg_output.has_message():
                self.validate_button.set_looks_warning(f"{self.validate_button_title} : {msg_config.get('public_data', 'copy_failed')}")
                return
            else:
                public_dir_path = os.path.join(get_data_dir(self.working_path), 'public_data')
                file_path = JsonFile(os.path.join(public_dir_path, 'validate_result.json'))
                file_path.write(data)
                self.validate_button.set_looks_success(f"{self.validate_button_title} : {msg_config.get('public_data', 'copy_done')}")
                self.validate_button.disabled = False

        except FileNotFoundError:
            message = msg_config.get('dg_web', 'missing_validate')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
            self.validate_button.set_looks_warning(f"{self.validate_button_title} : {msg_config.get('public_data', 'copy_failed')}")
            return

    @TaskDirector.callback_form('ノートブックをコピーする')
    def copy_notebooks(self, event):
        """実行済みタスクのノートブックをコピーするボタンが押下されたときに実行するメソッドです。"""

        self.notebooks_button.disabled = True
        self.notebooks_button.set_looks_processing(msg_config.get('public_data', 'copy_doing'))

        # 共通部分のdata_governance/researchflowまで(status.json)
        base_path = os.path.join(self.abs_root, path_config.DG_RESEARCHFLOW_FOLDER)

        # コピー元のNotebookファイルが存在するディレクトリ(workingディレクトリ)
        source_dir = os.path.dirname(os.path.dirname(self.working_path))
        old_part= os.path.join("publication", self.subflow_id)

        # 保存先パスの共通部分
        public_dir_path = os.path.join(get_data_dir(self.working_path), 'public_data')

        # 研究準備フェーズ
        ## status.jsonの内容を取得
        plan_path = os.path.join(base_path, 'plan', 'status.json')
        tasks = SubflowStatusFile(plan_path).read().tasks
        ## コピー元の実行済みNotebookファイルのディレクトリ
        working_task_dir = source_dir.replace(old_part, 'plan')
        ## 研究準備フェーズの保存先パス
        plan_destination_dir = os.path.join(public_dir_path, 'plan')
        # 実行済みのタスクのみをコピー
        for task in tasks:
            if task.completed_count > 0:
                utils._copy_file_by_name(task.name, working_task_dir, plan_destination_dir)

        # 実験フェーズ
        for subflow_name, subflow_id in self.experiment_subflow.items():
            ## status.jsonの内容を取得
            experiment_path = os.path.join(base_path, 'experiment', subflow_id, 'status.json')
            tasks = SubflowStatusFile(experiment_path).read().tasks
            ## コピー元の実行済みNotebookファイルのディレクトリ
            new_part = os.path.join("experiment", subflow_id)
            working_task_dir = source_dir.replace(old_part, new_part)
            ## 実験フェーズの保存先パス
            experiment_destination_dir = os.path.join(public_dir_path, 'experiment', subflow_name)
            ## 実行済みのタスクのみをコピー
            for task in tasks:
                if task.completed_count > 0:
                    utils._copy_file_by_name(task.name, working_task_dir, experiment_destination_dir)

        # 論文執筆フェーズ
        for subflow_name, subflow_id in self.writing_subflow.items():
            ## status.jsonの内容を取得
            writing_path = os.path.join(base_path, 'writing', subflow_id, 'status.json')
            tasks = SubflowStatusFile(writing_path).read().tasks
            ## コピー元の実行済みNotebookファイルのディレクトリ
            new_part = os.path.join("writing", subflow_id)
            working_task_dir = source_dir.replace(old_part, new_part)
            ## 論文執筆フェーズの保存先パス
            writing_destination_dir = os.path.join(public_dir_path, 'writing', subflow_name)
            ## 実行済みのタスクのみをコピー
            for task in tasks:
                if task.completed_count > 0:
                    utils._copy_file_by_name(task.name, working_task_dir, writing_destination_dir)

        # 査読フェーズ
        for subflow_name, subflow_id in self.review_subflow.items():
            ## status.jsonの内容を取得
            review_path = os.path.join(base_path, 'review', subflow_id, 'status.json')
            tasks = SubflowStatusFile(review_path).read().tasks
            ## コピー元の実行済みNotebookファイルのディレクトリ
            new_part = os.path.join("review", subflow_id)
            working_task_dir = source_dir.replace(old_part, new_part)
            ## 査読フェーズの保存先パス
            review_destination_dir = os.path.join(public_dir_path, 'review', subflow_name)
            # 実行済みのタスクのみをコピー
            for task in tasks:
                if task.completed_count > 0:
                    utils._copy_file_by_name(task.name, working_task_dir, review_destination_dir)

        self.notebooks_button.disabled = False
        self.notebooks_button.set_looks_success(f"{self.notebooks_button_title} : {msg_config.get('public_data', 'copy_done')}")

    async def get_data(self, token: str, project_id: str, remote_path: str) -> Optional[dict]:
        """指定したファイルの内容を取得するメソッドです。

        Args:
            token(str):パーソナルアクセストークン
            project_id(str):プロジェクトID
            remote_path(str):リモート先のパス

        Returns:
            Optional[dict]:ファイルの内容の値を返す。
        """
        data = None
        try:
            data = await self.grdm.download_json_file(
                token=token, base_url=self.grdm_url,
                project_id=project_id, remote_path=remote_path
            )
        except FileNotFoundError:
            raise FileNotFoundError
        except json.JSONDecodeError:
            message = msg_config.get('public_data', 'file_not_read')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception as e:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return data

CreatePublicFolder(os.path.abspath('__file__')).generate_buttons()

### 2. コピーしたデータを整理する
コピーしたデータを公開するのに適したデータとなるように整理してください。<br>
以下のセルを実行して公開データを表示し、公開に適したデータになるように整理を行ってください。

In [ ]:
# 公開データフォルダを表示する
import os

import panel as pn
from IPython.core.display import Javascript
from IPython.display import display

from library.utils.access import open_data_folder


folder_name = 'public_data'
button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 公開データにメタデータを付与する
公開データのメタデータ登録を行ってください。<br>
以下のセルを実行してメタデータの登録を行った後、このタスクに戻り作業を再開してください。<br>
GakuNin RDMにメタデータを登録する場合はGakuNin RDM上から手動で行ってください。<br>

In [ ]:
# メタデータを登録するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_register_metadata():
    """メタデータを登録するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/metadata.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_register_metadata_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_register_metadata()

## 再現性を確認する
メタデータの検証と再現性の確認を行ってください。<br>
<br>
※ 再現性を確認する機能は現在開発中です。

### メタデータを検証する
論拠データのメタデータの検証を行ってください<br>
セルを実行してメタデータの検証を行った後、このタスクに戻り作業を再開してください。<br>

In [ ]:
# 検証するタスクへアクセスするボタンを表示する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import message as msg_config
from library.utils.html.button import create_button


def access_validate():
    """検証するタスクへアクセスするボタンを表示するメソッドです。"""
    button_width = 500
    url = '../common/validate.ipynb'
    obj = create_button(
        url=f'{url}?init_nb=true',
        target='_blank',
        msg=msg_config.get('task', 'access_validate_task'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## GakuNin RDMに保存する

In [ ]:
# GakuNin RDMに保存する
import os
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.utils.config import path_config
from library.task_director import TaskDirector
from library.utils.setting import get_data_dir

script_file_name = 'organize_public_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。

    """
    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("5")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新

        # フォーム定義
        data_dir = get_data_dir(self.working_path)
        source = [os.path.join(data_dir, 'public_data')]
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()